In [40]:
import re
# import collections
import random, math
import numpy as np
from itertools import islice


# Testing

In [23]:
text = "Fresh breath and shining teeth enhance your personality."
# text = "We never attempt to get the reaction converted to completion because then the residence time , space time required in the reactor will be very , very high because it ' s a reversible reaction ."
# text = "When we say higher or less than the Km value , when we compare substrate concentration with the Km value , what we are looking at in the case of zero order regime , we often look at the Km value ; K ' m is usually less than 0.01 S0 ."
# text = re.sub('\\s+([.,!;?\'])\\s*', r'\1 ', text)
# text = re.sub('(\'\s*)', r"'", text)
# text = re.sub(r"n't", "not", text)
# print(text)
print(text[:-1])

Fresh breath and shining teeth enhance your personality


# Code

In [41]:
tags = ('RD_PUNC', 'V_VAUX', 'V_VM', 'N_NN', 'N_NNV', 'QT_QTC', 'PSP', 'CC_CCD', 'DM_DMD', 'RP_RPD', 'RP_NEG', 'JJ')

In [42]:
def english_match(strg, search=re.compile(r'^[a-zA-Z]').search):
    return bool(search(strg))

def pre_processing(text):
    text = text[:-1]
    #Remove space  around punc
    text = re.sub('\\s+([.,!;?\'])\\s*', r'\1 ', text)
    #Possession
    text = re.sub('(\'\s*)', r"'", text)

    text = re.sub(r"n't", "not", text)
    text = re.sub(r"there's", "there is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"K'm", "Km", text)
    #Adding Space around punc
    if text[-1] in ".,!?":
        text = text[:-1]+" "+text[-1]
    text = re.sub('([^0-9])([.,!?])([^0-9])', r'\1 \2 \3', text)

    return text

def tokenize(text):
    text = text.strip()
    tokens = text.split()
    tokens = [word.lower() for word in tokens]
    return tokens

def cleanup_text(text):
    pass

def slicing(sequence, n):
    it = iter(sequence)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

def flatten(sequences):
    if isinstance(sequences, (list,tuple)):
        return [a for i in sequences for a in flatten(i)]
    else:
        return [sequences]

In [94]:
class LanguageModel:
    def __init__(self, n = 4, smoothing = "w"):
        self.ngram = n
        self.vocabulary = set()
        self.count = {}
        self.smoothing = smoothing
        self.alpha = 0.001

    def train(self, tokens):
        self.vocabulary = set(flatten(tokens))
        self.vocabulary.add(None)
        for item in tokens:
            # print(item)
            for subitem in self.generate_ngrams(item):
                # print(subitem)
                if subitem[:self.ngram-1] not in self.count:
                    self.count[subitem[:self.ngram-1]] = {subitem[self.ngram-1:][0]: 1}
                else:
                    if subitem[self.ngram-1:][0] in self.count[subitem[:self.ngram-1]]:
                        self.count[subitem[:self.ngram-1]][subitem[self.ngram-1:][0]] += 1
                    else:
                        self.count[subitem[:self.ngram-1]][subitem[self.ngram-1:][0]] = 1
            # print(self.count[subitem[:self.ngram-1]])
            # break

        # print(len(self.count))
        # print(len(self.vocabulary))
        # print(self.count)

    def generate_ngrams(self, tokens):
        sequence = []
        if not tokens:
            return sequence
        if self.ngram!=1:
            tokens = [None]*(self.ngram-1) + tokens + [None]*(self.ngram-1)
        else:
            tokens = tokens + [None]
        
        result = slicing(tokens, self.ngram)
        for r in result:
            sequence.append(r)
        return sequence

    def calculate_probability(self, prev_ngram, unigram):
        if prev_ngram in self.count:
            if unigram in self.count[prev_ngram]:
                count = self.count[prev_ngram][unigram]
                total = sum(self.count[prev_ngram].values())
            else:
                count = 1
                total = sum(self.count[prev_ngram].values()) + 1

            prob = (count+self.alpha)/ (total+ (len(self.vocabulary)*self.alpha))

        else:
            count, total = 0, 1
            prob = random.random() / len(self.vocabulary)
        return prob

    def calculate_perplexity(self, tokens):
        perplexity, num = 0, 0
        for item in tokens:
            for subitem in self.generate_ngrams(item):
                num+= 1
                perplexity += math.log(self.calculate_probability(subitem[:self.ngram-1], subitem[self.ngram-1:][0]), 2)
        # print(perplexity)
        perplexity = math.pow(2, -1*(perplexity/num))
        return perplexity

    def find_val(self, token):
        if len(token) == 0:
            return sum(self.count[token].values())
        if token in self.count[token]:
            return self.count[token][token]
        else:
             return 0

    def witten2(self, token):
        uniq = 0
        for item in self.count:
            if token in item:
                uniq += 1
        k = uniq + find_val(token)
        if (k != 0 ):
            return uniq/k
        else:
            random.uniform(0.00001,0.0001)

    def witten(self, token):
        x = len(token)
        if  x==1:
            return find_val(token) / sum(self.count[token].values())
        if find_val(token[:-1]) != 0:
            return ((1 - self.witten2(token[:-1])) * (find_val(token) / find_val(token[:-1]))) + (self.witten2(token[:-1]) * self.witten(token[:-1]))
        else:
            return random.uniform(0.00001, 0.0001) +  (witten2(token[:-1]) * witten(token[:-1]))

    def kneyser(self, token, order):
        x = len(token)
        d = 0.75
        if find_val(token[:-1]) == 0:
            lamda = random.uniform(0,1)
            term = random.uniform(0.00001,0.0001)
        else:
            if x == order:
                term = max(0,find_val(token)-d)/find_val(token[:-1])
            else:
                term = max(0,(sum(token[1:] == token for token in ntokens_c[len(token) + 1].keys()) - d)) / find_val(token[:-1])
            lamda = d * sum(token[:-1] == token[:-1] for token in ntokens_c[len(token)].keys()) / find_val(token[:-1])
        if x == 1:
            return term
        else:
            return term + lamda * self.kneyser(token[:-1], maxi)



In [53]:
def prepare_data(path_corpus = ""):
    try:
        # fd_w = open("../Output/perplexity.txt","w")
        data = []
        with open(path_corpus,"r", encoding='ascii', errors = 'ignore') as fd_r:
            for line in fd_r:
                if line.startswith(tags) or not english_match(line):
                    continue
                text = pre_processing(line)
                data.append(tokenize(text))
                # fd_w.write(text + '\n')
        return data
    except Exception as e:
        print('Exception occurred while reading file', e.message)
        # fd_w.close()

In [77]:
data = prepare_data(path_corpus = "../Corpus/technical_domain_corpus.txt")
random.shuffle(data)

train_data, test_data = data[1000:], data[:1000]

lm = LanguageModel(4)
lm.train(train_data)

In [79]:
total_score, count = 0, 0
with open("../Output/2019201070-LM3-test-perplexity.txt","w") as fd_w:
    for item in test_data:
        score = lm.calculate_perplexity([item])
        line = ' '.join(item).capitalize()[:-2] +'.' + '\t' + str(score) + '\n'
        fd_w.write(line)
        total_score += score
        count += 1

    line = "\nAverage Score : " + str(total_score/count)
    fd_w.write(line)
# print(lm.calculate_perplexity(test_data))

# [test_data[0]]

In [68]:
print(lm.calculate_perplexity([test_data]))

28.194859825851736


In [70]:
print(' '.join(test_data[0]))

after understanding the lyrics the brain starts behaving differently with both .
